In [12]:
!conda list | grep scikit-learn

scikit-learn              1.5.0           py310h64e73be_1    conda-forge


In [13]:
!python -V

Python 3.10.13


In [14]:
import numpy as np
import pickle
import pandas as pd
from pathlib import Path

In [15]:
year = 2023
month = 3
taxi_type = 'yellow'

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

Path(f'output/{taxi_type}').mkdir(parents=True, exist_ok=True)

In [16]:
print('loading the model...')
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

loading the model...


In [17]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    return df

In [18]:
print(f'reading the data from {input_file}')
df = read_data(input_file)

reading the data from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet


In [19]:
print(f'applying the model ...')
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

applying the model ...


## Q1. What's the standard deviation of the predicted duration for this dataset?

In [20]:
np.std(y_pred)

6.247488852238703

## Q2. What's the size of the output file?

In [21]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [22]:
print(f'saving the result to {output_file} ...')
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

saving the result to output/yellow/2023-03.parquet ...


In [23]:
print(f'the output file have {Path(output_file).stat().st_size * (10**(-6))} Mb ...')

the output file have 68.64188 Mb ...


## Q3. Creating the scoring script

Now let's turn the notebook into a script. 

Which command you need to execute for that?

`jupyter nbconvert --to script score.ipynb`

## Q6. Docker container

In [1]:
!docker build -t ride-duration-prediction-service:v1 .

[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 414B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.2s
[+] Building 0.3s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 414B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.3s
[+] Building 0.5s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 414B                                       0.0s
 => [internal] load metadata for docker.

In [2]:
!docker run -it ride-duration-prediction-service:v1 2023 5

reading the data from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet
loading the model...
Preparing the data by applying the dictionary...
applying the model ...
The mean predicted duration for May is 0.19174419265916945
saving the result to output/yellow/2023-05.parquet ...
the output file have 70.390545 Mb ...
